# Reports - Buyers and Sellers

In [11]:
#Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import datetime
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [12]:
# iPython Settings
%matplotlib inline
sns.set(rc={'figure.figsize':(15,9)})
cf.go_offline()
init_notebook_mode(connected = True)
%matplotlib inline

# Creating Functions 

In [49]:
def createMilkData(records, accs):
    
    min_date = pd.to_datetime('2015-01-01')
    max_date = pd.to_datetime('2018-12-31')

    d = (max_date - min_date).days + 1

    df1_cols = ["shift", "acc no", "fat(3-10%)", "clr(23-30)", "amount", "rate", "payment"]
    df1_rows = [int(i) + 1 for i in range(records)]
    df1 = pd.DataFrame(np.random.randn(records, 7), df1_rows, df1_cols)
    df1['date'] = min_date + pd.to_timedelta(pd.np.random.randint(d,size=records), unit='d')
    df1['shift'] = df1["shift"].apply(lambda x : int(x)%2)
    df1['acc no'] = df1['acc no'].apply(lambda x : int(x*100)%accs + 1)
    df1['fat(3-10%)'] = df1['fat(3-10%)'].apply(lambda x : float(round((x*100)%8 + 3, 2)))
    df1['clr(23-30)'] = df1['clr(23-30)'].apply(lambda x : float(round((x*100)%8 + 23, 2)))
    df1['amount'] = df1['amount'].apply(lambda x : int((x*1000)%500 + 500))
    df1['snr'] = [int(i) for i in range (records)]
    df1['snr'] = df1['snr'].apply(lambda x : float(df1['fat(3-10%)'].iloc[x])/5 + float(df1['clr(23-30)'].iloc[x])/4.0 + 0.6)
    df1['rate'] = df1['rate'].apply(lambda x : round((x*100)%20 + 50, 2))
    df1['payment'] = [int(i) for i in range (records)]
    df1['payment'] = df1['payment'].apply(lambda x : df1['amount'].iloc[x]*df1['rate'].iloc[x])
    df1['debt'] = pd.DataFrame([random.randint(0, 30000) for i in range(records)])
    
    return df1

def createItemData(records, items, accs):
    
    min_date = pd.to_datetime('2015-01-01')
    max_date = pd.to_datetime('2018-12-31')

    d = (max_date - min_date).days + 1
    
    df_cols = ['acc no', 'item code', 'quantity', 'cost per unit', 'total amount', 'buy/sell']
    df_rows = [int(i) + 1 for i in range(records)]
    df = pd.DataFrame(np.random.randn(records, 6), df_rows, df_cols)
    df['date'] = min_date + pd.to_timedelta(pd.np.random.randint(d,size=records), unit='d')
    df['acc no'] = df['acc no'].apply(lambda x : int(x*100)%accs + 1)
    #print(df['item code'])
    df['item code'] = df['item code'].apply(lambda x : int(x*100)%items + 1)
    df['quantity'] = df['quantity'].apply(lambda x : int((x*1000)%500 + 500))
    #df['cost per unit'] = df['item code']
    #df['cost per unit'] = df['item code'].apply(lambda x : int(x))
    
    items_list = [i + 1 for i in range(items)]
    rate_list = [random.randint(50, 100) for i in range(items)]
    ItemTranslation = dict(zip(items_list, rate_list))
    df['cost per unit'] = df['item code'].apply(lambda x : ItemTranslation[x])
    
    df['total amount'] = [int(i) for i in range (records)] 
    df['total amount'] = df['total amount'].apply(lambda x : round(df['quantity'].iloc[x]*df['cost per unit'].iloc[x], 2))
    df['buy/sell'] = df["buy/sell"].apply(lambda x : int(x)%2)
    df['debt'] = pd.DataFrame([random.randint(0, 30000) for i in range(records)])
    
    return df

def filterByDate(df1, sd, sm, sy, ed, em, ey):
    
    return df1[(df1['date']>datetime.date(sy,sm,sd)) & (df1['date']<datetime.date(ey,em,ed))] 

def stdError(df, column):
    #mean +- std/sqrt(count)
    mean = df[column].describe()['mean']
    std_int = df[column].describe()['std']/(df[column].describe()['count']**0.5)
    
    return (int(mean - std_int), int(mean + std_int))

def sellerReportMilkGranular(df1, sd, sm, sy, ed, em, ey, acc_no):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['acc no'] == int(acc_no)]
    snr = stdError(df, 'snr')
    total_amt = df['amount'].sum()
    avg_amount = stdError(df, 'amount')
    #iters = df.describe()['count']
    morning_shift = df[df['shift'] == 0]['shift'].describe()['count']
    evening_shift = df[df['shift'] == 1]['shift'].describe()['count']
    iters = morning_shift + evening_shift
    avg_debt = stdError(df, 'debt')
    avg_rate = stdError(df, 'rate')
    total_pay = df['payment'].sum()
    avg_payment = stdError(df, 'payment')
    
    return f"""
    Account Number: {acc_no}\n
    SNF: {snr}\n
    Total Amount: {total_amt}\n
    Average Amount: {avg_amount}\n
    Total Visits: {iters}\n
    Morning Shifts: {morning_shift}\n
    Evening Shifts: {evening_shift}\n
    Average Debt: {avg_debt}\n
    Average Rate: {avg_rate}\n
    Total Payment: {total_pay}\n
    Average Payment: {avg_payment}\n
    
    """
    
    #return [snr, (total_amt, avg_amount), (iters, morning_shift, evening_shift), avg_debt, avg_rate, (total_pay, avg_payment)]
    

def sellerReportMilk(df1, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    snr = stdError(df, 'snr')
    total_amt = df['amount'].sum()
    avg_amount = stdError(df, 'amount')
    #iters = df.describe()['count']
    morning_shift = df[df['shift'] == 0]['shift'].describe()['count']
    evening_shift = df[df['shift'] == 1]['shift'].describe()['count']
    iters = morning_shift + evening_shift
    avg_debt = stdError(df, 'debt')
    avg_rate = stdError(df, 'rate')
    total_pay = df['payment'].sum()
    avg_payment = stdError(df, 'payment')
    
    return f"""
    SNF: {snr}\n
    Total Amount: {total_amt}\n
    Average Amount: {avg_amount}\n
    Total Visits: {iters}\n
    Morning Shifts: {morning_shift}\n
    Evening Shifts: {evening_shift}\n
    Average Debt: {avg_debt}\n
    Average Rate: {avg_rate}\n
    Total Payment: {total_pay}\n
    Average Payment: {avg_payment}\n
    
    """
    
    #return [snr, (total_amt, avg_amount), (iters, morning_shift, evening_shift), avg_debt, avg_rate, (total_pay, avg_payment)]



def sellerReportMilkByShiftVisualGranular(df, acc_no):
    #sns.countplot(x = 'shift', data = df[df['acc no'] == int(acc_no)])
    
    return None

def sellerReportMilkByCount(df):
    #sns.countplot(x = 'acc no', data = df)
    
    return None

def passbookRecordsMilk(df1, acc_no, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['acc no'] == int(acc_no)]
    df = df.sort_values('date')
    df.drop(columns = 'acc no', inplace = True)
    
    records = [tuple(x) for x in df.to_records(index=False)]
    
    return df
    #return records

def visualizeMilkGranular(df1, acc_no, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['acc no'] == int(acc_no)]
    df.sort_values('date')
    
    #df['amount'].iplot(kind = 'hist')
    df['payment'].iplot(kind = 'line')
    df[['fat(3-10%)', 'clr(23-30)']].iplot(kind = 'box')
    df.iplot(kind = "scatter", x = 'snr', y = "rate", mode = "markers")
    #df.iplot(kind = 'bar', x = 'shift', y = 'amount')
    
    return None

def visualizeMilk(df1, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df.sort_values('date')
    #df['amount'].iplot(kind = 'hist')
    df['payment'].iplot(kind = 'line')
    df[['fat(3-10%)', 'clr(23-30)']].iplot(kind = 'box')
    df.iplot(kind = "scatter", x = 'snr', y = "rate", mode = "markers")
    #df.iplot(kind = 'bar', x = 'acc no', y = 'amount')
    
    return None

def sellerReportItemsGranular(df1, sd, sm, sy, ed, em, ey, acc_no):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['acc no'] == int(acc_no)]
    total_amt = df['quantity'].sum()
    avg_amount = stdError(df, 'quantity')
    #iters = df.describe()['count']
    buy = df[df['buy/sell'] == 0]['buy/sell'].describe()['count']
    sell = df[df['buy/sell'] == 1]['buy/sell'].describe()['count']
    iters = buy + sell
    avg_debt = stdError(df, 'debt')
    avg_rate = stdError(df, 'cost per unit')
    total_purchased = df[df['buy/sell'] == 0]['buy/sell'].sum()
    total_sold = df[df['buy/sell'] == 1]['buy/sell'].sum()
    
    return f"""
    Account Number: {acc_no}\n
    Total Amount: {total_amt}\n
    Average Amount: {avg_amount}\n
    Total Visits: {iters}\n
    Items Brought: {buy}\n
    Items Sold: {sell}\n
    Average Debt: {avg_debt}\n
    Average Transaction Rate: {avg_rate}\n
    Total Entities Purchased: {total_purchased}\n
    Total Entities Sold: {total_sold}\n
    """
    
    #return [(total_amt, avg_amount), (iters, buy, sell), avg_debt, avg_rate, (total_purchased, total_sold)]

def sellerReportItems(df1, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    total_amt = df['quantity'].sum()
    avg_amount = stdError(df, 'quantity')
    #iters = df.describe()['count']
    buy = df[df['buy/sell'] == 0]['buy/sell'].describe()['count']
    sell = df[df['buy/sell'] == 1]['buy/sell'].describe()['count']
    iters = buy + sell
    avg_debt = stdError(df, 'debt')
    avg_rate = stdError(df, 'cost per unit')
    total_purchased = df[df['buy/sell'] == 0]['buy/sell'].sum()
    total_sold = df[df['buy/sell'] == 1]['buy/sell'].sum()
    
    return f"""
    Total Amount: {total_amt}\n
    Average Amount: {avg_amount}\n
    Total Visits: {iters}\n
    Items Brought: {buy}\n
    Items Sold: {sell}\n
    Average Debt: {avg_debt}\n
    Average Transaction Rate: {avg_rate}\n
    Total Entities Purchased: {total_purchased}\n
    Total Entities Sold: {total_sold}\n
    """
    
    #return [(total_amt, avg_amount), (iters, buy, sell), avg_debt, avg_rate, (total_purchased, total_sold)]

def passbookRecordsItemByAccount(df1, acc_no, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    #df = df[df['acc no'] == int(acc_no)]
    df = df[df['acc no'] == acc_no]
    df.sort_values('date')
    df.drop(columns = acc_no, inplace = True)
    
    records = [tuple(x) for x in df.to_records(index=False)]
    
    return df
    #return records

def passbookRecordsItemByItem(df1, item_code, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['item code'] == int(item_code)]
    df.sort_values('date')
    df.drop(columns = item_code, inplace = True)
    
    records = [tuple(x) for x in df.to_records(index=False)]
    
    return df
    #return records

def visualizeItemGranularByItemID(df1, item_code, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['item code'] == int(item_code)]
    df.sort_values('date')
    
    df.iplot(kind = 'bar', x = 'acc no', y = 'quantity')
    #df[df['buy/sell'] == 0].iplot(kind = 'bar', x = 'acc no', y = 'quantity')
    #df[df['buy/sell'] == 1].iplot(kind = 'bar', x = 'acc no', y = 'quantity')
    
    return None

def visualizeItemGranularByAccId(df1, acc_no, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df = df[df['acc no'] == int(acc_no)]
    df.sort_values('date')
    
    df.iplot(kind = 'bar', x = 'item code', y = 'quantity')
    #df[df['buy/sell'] == 0].iplot(kind = 'bar', x = 'item code', y = 'quantity')
    #df[df['buy/sell'] == 1].iplot(kind = 'bar', x = 'item code', y = 'quantity')
    
def visualizeItems(df1, sd, sm, sy, ed, em, ey):
    
    df = filterByDate(df1, sd, sm, sy, ed, em, ey)
    df.sort_values('date')
    
    df['total amount'].iplot(kind = 'line')
    #df[df['buy/sell'] == 0]['total amount'].iplot(kind = 'line')
    #df[df['buy/sell'] == 1]['total amount'].iplot(kind = 'line')
    
    
    
    
    
    
    

In [14]:
items = createItemData(200, 5, 20)
milk = createMilkData(200, 10)



In [18]:
items.head()

,acc no,item code,quantity,cost per unit,total amount,buy/sell,date,debt
1,13,3,688,94,64672,0,2016-05-07,21560.0
2,11,2,658,87,57246,1,2015-01-15,15916.0
3,18,4,737,84,61908,0,2015-08-15,20682.0
4,5,4,904,84,75936,0,2015-12-13,28086.0
5,10,5,808,63,50904,1,2015-10-18,15669.0


In [19]:
milk.head()

,shift,acc no,fat(3-10%),clr(23-30),amount,rate,payment,date,snr,debt
1,0,10,6.68,27.40,799,58.52,46757.48,2016-11-27,8.7860,12057.0
2,0,4,6.03,26.07,818,60.73,49677.14,2016-05-13,8.3235,9169.0
3,0,10,4.57,26.01,936,65.97,61747.92,2017-11-07,8.0165,3404.0
4,0,2,6.65,28.71,911,58.15,52974.65,2018-06-24,9.1075,1558.0
5,1,5,4.83,25.21,603,52.84,31862.52,2017-03-18,7.8685,18510.0


In [16]:
print(sellerReportMilkGranular(milk, 1, 1, 2017, 31, 12, 2017, 1))


    Account Number: 1

    SNR: (8, 8)

    Total Amount: 4220

    Average Amount: (648, 758)

    Total Visits: 6.0

    Morning Shifts: 4.0

    Evening Shifts: 2.0

    Average Debt: (13832, 19546)

    Average Rate: (58, 65)

    Total Payment: 261460.87999999998

    Average Payment: (39772, 47381)

    
    


/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [19]:
print(sellerReportMilk(milk, 1, 1, 2017, 31, 12, 2017))


    SNF: (8, 8)

    Total Amount: 38116

    Average Amount: (715, 750)

    Total Visits: 52.0

    Morning Shifts: 44.0

    Evening Shifts: 8.0

    Average Debt: (13819, 16055)

    Average Rate: (60, 61)

    Total Payment: 2313132.6200000006

    Average Payment: (43323, 45643)

    
    


/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [27]:
print(passbookRecordsMilk(milk, 1, 1, 1, 2017, 31, 12, 2017))

     shift  fat(3-10%)  clr(23-30)  amount   rate   payment       date  \
43       1        6.70       29.76     856  69.81  59757.36 2017-02-25   
190      0        9.69       26.93     516  66.62  34375.92 2017-03-28   
19       0        3.74       29.47     826  50.37  41605.62 2017-04-05   
33       0        9.04       25.90     758  63.94  48466.52 2017-10-08   
35       1        4.44       23.14     591  69.33  40974.03 2017-10-24   
15       0        7.60       25.01     673  53.91  36281.43 2017-10-30   

        snr     debt  
43   9.3800   8415.0  
190  9.2705  20138.0  
19   8.7155  16836.0  
33   8.8830   9167.0  
35   7.2730  26877.0  
15   8.3725  18704.0  


/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [35]:
visualizeMilkGranular(milk, 1, 1, 1, 2017, 31, 12, 2017)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [36]:
visualizeMilk(milk, 1, 1, 2017, 31, 12, 2017)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [38]:
print(sellerReportItemsGranular(items, 1, 1, 2017, 31, 12, 2017, 1))


    Account Number: 1

    Total Amount: 2209

    Average Amount: (641, 831)

    Total Visits: 3.0

    Items Brought: 2.0

    Items Sold: 1.0

    Average Debt: (4225, 12383)

    Average Transaction Rate: (81, 87)

    Total Entities Purchased: 0

    Total Entities Sold: 1

    


/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [39]:
print(sellerReportItems(items, 1, 1, 2017, 31, 12, 2017))


    Total Amount: 37418

    Average Amount: (742, 784)

    Total Visits: 49.0

    Items Brought: 35.0

    Items Sold: 14.0

    Average Debt: (14359, 16844)

    Average Transaction Rate: (72, 77)

    Total Entities Purchased: 0

    Total Entities Sold: 14

    


/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [51]:
#print(passbookRecordsItemByAccount(items, 13, 1, 1, 2017, 31, 12, 2017))
#passbookRecordsItemByItem(items, 1, 1, 1, 2017, 31, 12, 2017)

In [43]:
visualizeItemGranularByItemID(items, 1, 1, 1, 2017, 31, 12, 2017)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [44]:
visualizeItemGranularByAccId(items, 1, 1, 1, 2017, 31, 12, 2017)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [45]:
visualizeItems(items, 1, 1, 2017, 31, 12, 2017)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.

